In [3]:
import bz2
import numpy as np
from tqdm import tqdm_notebook as tqdm
import gzip
from heapq import heappushpop
from joblib import Parallel, delayed
import time

In [4]:
import numpy as np
import os

classes = []

for root, dirs, files in os.walk("."):  
    for filename in files:
        classes.append(filename)

In [5]:
from re import compile

rex = compile('n[0-9]+')
classes = [x for x in classes if rex.match(x)]

In [6]:
x_trn = []
y_trn = []
for c in tqdm(classes):
    x_trn.append(np.load(c))
    y_trn += [c] * len(x_trn[-1])

In [7]:
x_trn = np.vstack(x_trn)
print(len(x_trn))
print(len(y_trn))

997659
997659


In [8]:
from sklearn.utils import shuffle
x_trn, y_trn = shuffle(x_trn, y_trn, random_state=0)

In [9]:
x_trn = np.reshape(x_trn, (-1, 2048))
x_tst, y_tst = x_trn[:100], y_trn[:100]
x_val, y_val = x_trn[100:1100], y_trn[100:1100]
x_trn, y_trn = x_trn[1100:], y_trn[1100:]

In [9]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(x_trn, y_trn)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [10]:
neigh.score(x_tst, y_tst)

0.8

In [11]:
neigh.score(x_val, y_val)

0.681

In [8]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',
                         multi_class='multinomial', n_jobs=-1).fit(x_trn, y_trn)

In [9]:
clf.score(x_tst, y_tst)

0.9

In [10]:
clf.score(x_val, y_val)

0.81

In [11]:
clf.score(x_trn, y_trn)

0.9549760726660439

In [12]:
len(classes)

1000

In [14]:
classes2idx = {}
for i, x in enumerate(classes):
    classes2idx[x] = i

In [15]:
y_trn_idx = [classes2idx[x] for x in y_trn]

In [17]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="6"

from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(len(classes), input_dim=2048))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(x_trn, y_trn_idx, epochs=5, batch_size=32)

Epoch 1/5
996559/996559 [==============================] - 134s 135us/step - loss: 1.2208 - acc: 0.7336
Epoch 2/5
996559/996559 [==============================] - 123s 123us/step - loss: 0.6757 - acc: 0.8191
Epoch 3/5
996559/996559 [==============================] - 122s 122us/step - loss: 0.5928 - acc: 0.8383
Epoch 4/5
996559/996559 [==============================] - 125s 125us/step - loss: 0.5440 - acc: 0.8506
Epoch 5/5
996559/996559 [==============================] - 122s 122us/step - loss: 0.5092 - acc: 0.8597


In [18]:
y_tst_idx = [classes2idx[x] for x in y_tst]
y_val_idx = [classes2idx[x] for x in y_val]

In [19]:
model.evaluate(x_tst, y_tst_idx, batch_size=128)

100/100 [==============================] - 0s 384us/step


[0.3620190918445587, 0.9100000262260437]

In [20]:
model.evaluate(x_val, y_val_idx, batch_size=128)

1000/1000 [==============================] - 0s 35us/step


[0.6134206886291504, 0.8280000019073487]

In [22]:
model.fit(x_trn, y_trn_idx, epochs=5, batch_size=32)

Epoch 1/5
996559/996559 [==============================] - 122s 123us/step - loss: 0.4698 - acc: 0.8701
Epoch 2/5
996559/996559 [==============================] - 123s 124us/step - loss: 0.4491 - acc: 0.8758
Epoch 3/5
996559/996559 [==============================] - 121s 122us/step - loss: 0.4316 - acc: 0.8806
Epoch 4/5
996559/996559 [==============================] - 121s 121us/step - loss: 0.4163 - acc: 0.8849
Epoch 5/5
996559/996559 [==============================] - 121s 122us/step - loss: 0.4026 - acc: 0.8889


In [23]:
model.evaluate(x_tst, y_tst_idx, batch_size=128)

100/100 [==============================] - 0s 33us/step


[0.33740347623825073, 0.9100000262260437]

In [24]:
model.evaluate(x_val, y_val_idx, batch_size=128)

1000/1000 [==============================] - 0s 42us/step


[0.5951442904472352, 0.8239999980926513]

In [25]:
model.fit(x_trn, y_trn_idx, epochs=5, batch_size=32)

Epoch 1/5
996559/996559 [==============================] - 126s 126us/step - loss: 0.3904 - acc: 0.8924
Epoch 2/5
996559/996559 [==============================] - 123s 123us/step - loss: 0.3790 - acc: 0.8960
Epoch 3/5
996559/996559 [==============================] - 115s 116us/step - loss: 0.3687 - acc: 0.8989
Epoch 4/5
996559/996559 [==============================] - 115s 116us/step - loss: 0.3591 - acc: 0.9015
Epoch 5/5
996559/996559 [==============================] - 124s 124us/step - loss: 0.3503 - acc: 0.9043


In [26]:
model.evaluate(x_tst, y_tst_idx, batch_size=128)

100/100 [==============================] - 0s 49us/step


[0.34398719668388367, 0.8999999761581421]

In [27]:
model.evaluate(x_val, y_val_idx, batch_size=128)

1000/1000 [==============================] - 0s 30us/step


[0.5886992530822754, 0.8349999971389771]

In [10]:
from sklearn.neighbors import KNeighborsClassifier
neigh2 = KNeighborsClassifier(n_neighbors=2)
neigh2.fit(x_trn, y_trn)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform')

In [11]:
neigh2.score(x_tst, y_tst)

0.81

In [12]:
neigh2.score(x_val, y_val)

0.67

In [13]:
from sklearn.neighbors import KNeighborsClassifier
neigh5 = KNeighborsClassifier(n_neighbors=5)
neigh5.fit(x_trn, y_trn)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [14]:
neigh5.score(x_tst, y_tst)

0.83

In [15]:
neigh5.score(x_val, y_val)

0.735